In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# from sklearn.ensemble import RadomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
print("data size:", df.shape)
df.head()

In [ ]:
df.loc[df['Sex'] == 'male', 'Sex_male'] = 1
df.loc[df['Sex'] == 'female', 'Sex_female'] = 1
df['Sex_male'].fillna(0, inplace=True)
df['Sex_female'].fillna(0, inplace=True)
df.drop(['Name', 'Sex', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1, inplace=True)
df.describe()

In [ ]:
print(df.nunique())

In [ ]:
print("Age uniquw value:", df['Age'].unique())


In [ ]:
print(df.isnull().sum(), '\n')
df['Age'].fillna(int(df['Age'].mean()), inplace=True)
#df.dropna(how='any', axis=0, inplace=True)
print(df.isnull().sum())

In [ ]:
survived1_df = df[df['Survived'] == 1].copy()
survived0_df = df[df['Survived'] == 0].copy()

hist_list = [survived0_df['Pclass'], survived0_df['Sex_male'], survived0_df['Sex_female'],
             survived0_df['Age'], survived0_df['SibSp'], survived0_df['Parch'], 
             survived1_df['Pclass'], survived1_df['Sex_male'], survived1_df['Sex_female'], 
             survived1_df['Age'], survived0_df['SibSp'], survived1_df['Parch']]

fig1, axes1 = plt.subplots(2, 6, subplot_kw={'xlabel':'x axis', 'ylabel':'y axis'}, figsize=(28, 12))

for i, (ax, hist) in enumerate(zip(axes1.ravel(), hist_list)):
    ax.hist(hist)
    ax.set_title('data' + str(i+1))
    
plt.tight_layout()
plt.show()

In [ ]:
#mms = MinMaxScaler()
#sds = StandardScaler()
#mms.fit(df.iloc[:, 2:].values)
#sds.fit(df.iloc[:, 2:].values)

#train_data = mms.transform(df.iloc[:, 2:].values)
train_data = df.iloc[:, 2:].values
train_label = df.iloc[:, 1].values

#X_train, X_validation, Y_train, Y_validation = train_test_split(train_data, train_label, random_state=0)
param_list = [0.001, 0.01, 0.1, 1, 10, 100]
param_grid = {'C': param_list, 'gamma': param_list}

grid_search = GridSearchCV(SVC(), param_grid, cv=len(param_list))
grid_search.fit(train_data, train_label)

print("1st validation best score: {}".format(grid_search.best_score_))
print("1st validation best param: {}".format(grid_search.best_params_))

In [ ]:
best_gamma = grid_search.best_params_['gamma']
best_C = grid_search.best_params_['C']

gamma_list = [best_gamma*i for i in range(1,11)]
C_list = [best_C*i for i in range(1,11)]

param_grid = {'C': C_list, 'gamma': gamma_list}

grid_search = GridSearchCV(SVC(), param_grid, cv=len(C_list))
grid_search.fit(train_data, train_label)

print("2nd validation best score: {}".format(grid_search.best_score_))
print("2nd validation best param: {}".format(grid_search.best_params_))

In [ ]:
best_gamma = grid_search.best_params_['gamma']
best_C = grid_search.best_params_['C']

gamma_list = [best_gamma + j*best_gamma*i/10 for i in range(1,10) for j in range(-1, 2, 2)]
C_list = [best_C + j*best_C*i/10 for i in range(1,10) for j in range(-1, 2, 2)]

param_grid = {'C': C_list, 'gamma': gamma_list}

grid_search = GridSearchCV(SVC(), param_grid, cv=len(C_list))
grid_search.fit(train_data, train_label)

print("3rd validation best score: {}".format(grid_search.best_score_))
print("3rd validation best param: {}".format(grid_search.best_params_))

In [ ]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')

test.loc[test['Sex'] == 'male', 'Sex_male'] = 1
test.loc[test['Sex'] == 'female', 'Sex_female'] = 1
test['Sex_male'].fillna(0, inplace=True)
test['Sex_female'].fillna(0, inplace=True)
test.drop(['Name', 'Sex', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1, inplace=True)
print(test.isnull().sum(), '\n')

In [ ]:
test['Age'].fillna(int(test['Age'].mean()), inplace=True)
print(df.isnull().sum())

In [ ]:
test.head()

In [ ]:
test_data = test.iloc[:, 1:].values

clf = SVC(gamma=grid_search.best_params_['gamma'], C=grid_search.best_params_['C'])
clf.fit(train_data, train_label)
pred = clf.predict(test_data)

In [ ]:
PassengerId = np.array(test['PassengerId']).astype(int)
df_pred = pd.DataFrame(pred, PassengerId, columns=['Survived'])
df_pred.to_csv('my_svm.cdv', index_label=['PassengerId'])